In [1]:
import json
import os

with open('/home/jovyan/work/shared/hasham/jsl_keys.json', 'r') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [2]:
import pandas as pd
import requests
import json
from zipfile import ZipFile
from io import BytesIO
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 3.4.1
Spark NLP_JSL Version : 3.4.1


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-72e25829-888e-4467-9cec-3c67d224d99f;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.1 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-jav

In [3]:
from sparknlp_jsl.training import *
from sparknlp.training import *
from pyspark.sql import functions as F

json_path = './result_new.json'

rdr = AnnotationToolJsonReader(assertion_labels = ['ADE_Content'])

df_anns = rdr.readDataset(spark, json_path).withColumn("json",F.lit(json_path))


In [4]:
def df_to_conll (json_df, project_id):
    df = json_df.select("json","task_id",F.explode(F.arrays_zip('token.begin','token.end','token.result','ner_label.result',"token.metadata")).alias("cols")) \
    .select("json","task_id",
            F.expr("cols['0']").alias("begin"),
            F.expr("cols['1']").alias("end"),
            F.expr("cols['2']").alias("token"),
            F.expr("cols['3']").alias("ner"),
           F.expr("cols['4'].sentence").alias("sentence")).toPandas()
    
    conll_lines=["-DOCSTART- -X- -X- O\n\n"]
    for j,project in enumerate(df["json"].unique()):
        project_df = df[df["json"]==project].reset_index(drop=True)
        print ("project ", j)
        for t,task in enumerate(project_df.task_id.unique()):
            print (j, t)
            task_df = project_df[project_df.task_id==task].reset_index(drop=True)
            for sent in task_df.sentence.unique():
                #print (task, sent)
                sent_df = task_df[task_df.sentence==sent].sort_values(by=["begin"]).reset_index(drop=True)
                for i,row in sent_df.iterrows():
                    #print (task, sent, i)
                    conll_lines.append(row["token"]+" -X- -X- "+row["ner"]+"\n")
                #print ("end of sent ")
                conll_lines.append("\n")
    with open('./project_{}_ner.conll'.format(project_id), 'w') as f:
        for i in conll_lines:
            f.write(i)
    return './project_{}_ner.conll'.format(project_id)

df_to_conll(df_anns, 'demo')

!head -n 20 ./project_demo_ner.conll


project  0
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
-DOCSTART- -X- -X- O

February -X- -X- B-DATE
9 -X- -X- I-DATE
, -X- -X- I-DATE
2022 -X- -X- I-DATE
ClinicalTrials.gov -X- -X- O
Results -X- -X- O
Results -X- -X- O
First -X- -X- O
Posted -X- -X- O
: -X- -X- O
02/09/2022 -X- -X- B-DATE
Last -X- -X- O
Update -X- -X- O
Posted -X- -X- O
: -X- -X- O
02/09/2022 -X- -X- B-DATE
Recruitment -X- -X- O
Details -X- -X- O


In [ ]:


conll_data = CoNLL().readDataset(spark, 'project_demo_ner.conll') 

embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models')\
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings") 

ner = MedicalNerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(30)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True)\
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('./logs')\
    .setGraphFolder('./logs')\
    .setLogPrefix('ner_main')\
    .setValidationSplit(0.2)

ner_pipeline = Pipeline(stages = [embeddings, ner])
ner_model = ner_pipeline.fit(conll_data) #change


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ]embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[OK!]


22/04/28 21:22:02 WARN TaskSetManager: Stage 2 contains a task of very large size (2762 KiB). The maximum recommended task size is 1000 KiB.
22/04/28 21:22:06 WARN TaskSetManager: Stage 3 contains a task of very large size (2762 KiB). The maximum recommended task size is 1000 KiB.
2022-04-28 21:22:10.709763: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-28 21:22:10.815933: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /j

Training started - total epochs: 30 - lr: 0.001 - batch size: 8 - labels: 27 - chars: 116 - training examples: 4453
Epoch 1/30 started, lr: 0.001, dataset size: 4453
Epoch 1/30 - 84.61s - total training loss: 14326.667 - avg training loss: 25.6291 - batches: 559
Quality on validation dataset (20.0%), validation examples = 890


In [ ]:
from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("ner_dl")

tf_graph.build("ner_dl", build_params={"embeddings_dim": 200, 
                                       "nchars": 120, "ntags": 30, 
                                       "is_medical": 1}, 
               model_location="./logs", model_filename="auto")
